In [6]:
schools = ['American University of Nigeria', 'Augustine University', 'Baze University', 'Bells University of Technology', 'Benson Idahosa University']

for school in schools:
    # Load packages
    from datetime import datetime
    import time
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from bs4 import BeautifulSoup

    import pandas as pd

    # Set up requirements
    options = Options()

    options.headless = True

    driver_path = "C:/Users/WEB3/Downloads/Compressed/chromedriver_win32/chromedriver"
    browser = webdriver.Chrome(executable_path=driver_path, options=options)

    # Request for the url
    url = "https://scholar.google.com/"

    browser.get(url)

    text_field = browser.find_element_by_id('gs_hdr_tsi')

    text_field.send_keys(f'\"{school}\"')
    time.sleep(2)
    browser.find_element_by_id('gs_hdr_tsb').click()
    time.sleep(2)
    browser.find_element_by_id('gs_hdr_mnu').click()
    time.sleep(2)
    browser.find_element_by_xpath("//div[@class='gs_hdr_drw_sec']/a[3]").click()

    scholar_name = []
    scholar_link = []

    print(f'Extracting data for {school}')

    # Zoom in to a specific section of the page
    def get_page_items():
        scholars = browser.find_elements_by_xpath("//h3[@class='gs_ai_name']")

        # Extract relevant details
        for scholar in scholars:
            scholar_detail = scholar.find_element_by_tag_name('a')
            scholar_name.append(scholar_detail.text)
            scholar_link.append(scholar_detail.get_attribute('href'))

    pages = list(range(0,10000000)) # Estimated number of pages, here I use an impossible end value so as not to run out of range.
    # count = 0
    # Loop through the pages
    for page in pages:
        print(f"Scraping page: {page+1}...")
        # count += 1
        
        get_page_items()
        time.sleep(2)
        next_page = browser.find_element_by_xpath('//button[@aria-label="Next"]')

        if next_page.get_attribute('aria-label') == 'Next' and next_page.get_attribute('disabled') != 'true':
            next_page.click()
        else:
            break

        # if count == 1:
        #     break

    # Save extracted data to a dataframe
    scholars_table = pd.DataFrame({'faculty_name': scholar_name, 'profile_link': scholar_link})

    browser.quit()
    time.sleep(5)

    # Reload or refresh selenium to avoid break or timeout
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    # Set up requirements
    options = Options()

    options.headless = True

    driver_path = "C:/Users/WEB3/Downloads/Compressed/chromedriver_win32/chromedriver"
    browser = webdriver.Chrome(executable_path=driver_path, options=options)

    # Items to be extracted from each scholars profile page
    scholar_page_name = []
    bu_mail = []
    citations = []
    h_index = []
    i10_index = []
    citations_last_5_years = []
    h_index_last_5_years = []
    i10_index_last_5_years = []
    profile_url = []
    home_page_link = []
    total_publications = []
    area_of_specialization = []
    number_of_collaborators = []
    school_name = []

    num = 1
    # Go to each scholars profile page
    for link in scholars_table['profile_link']:
        browser.get(link)

        print(f'Extracting scholar: {num} data')
        # Get specializations
        specializations = browser.find_elements_by_xpath("//div[@id='gsc_prf_int']/a")

        area_specs = []
        for area in specializations:
            area = area.text
            area = area.replace(" ", "_")
            
            if "_and_" in area:
                area = area.split("_and_")
                for a in area:
                    area_specs.append(a)
            else:
                area_specs.append(area)

        area_of_specialization.append(" ".join(area_specs))

        # Loop through the show more button clicks
        for page in pages:
    
            total_articles = browser.find_elements_by_class_name('gsc_a_tr')
            
            time.sleep(2)
            try:
                next_page = browser.find_element_by_id('gsc_bpf_more')

                if next_page.get_attribute('disabled') != 'true':
                    next_page.click()
                    time.sleep(4)
                else:
                    break
            except:
                break
    
        total_publications.append(len(total_articles))

        html = browser.page_source

        soup = BeautifulSoup(html, 'html.parser')
        
        profile_url.append(link)

        scholar_page_name.append(soup.select_one('#gsc_prf_in').get_text())
        school_name.append(school)
        try:
            homepage_link = soup.select_one("#gsc_prf_ivh a")['href']
        except:
            homepage_link = None
        
        # Check if BU faculty Homepage link exists and extract mail user
        if homepage_link is not None:
            if homepage_link.split('/')[-2] != "faculty":
                bu_mail.append(None)
                home_page_link.append(None)
            else:
                email = homepage_link.split('/')[-1] + '@babcock.edu.ng'
                bu_mail.append(email)
                home_page_link.append(homepage_link)

            # user_id = email.split('@')[0]
            
            # if user_id != '':
            #     bu_mail.append(email)
            # else:
            #     bu_mail.append(None)
            
            # home_page_link.append(homepage_link)
        else:
            bu_mail.append(None)
            home_page_link.append(None)

        # Get citation summary table
        try:
            citations.append(int(soup.select_one('tr:nth-child(1) .gsc_rsb_sc1+ .gsc_rsb_std').get_text()))
            citations_last_5_years.append(int(soup.select_one('tr:nth-child(1) .gsc_rsb_std+ .gsc_rsb_std').get_text()))

            h_index.append(int(soup.select_one('tr:nth-child(2) .gsc_rsb_sc1+ .gsc_rsb_std').get_text())) # alternative to tr:nth-child(2) is tr~ tr
            h_index_last_5_years.append(int(soup.select_one('tr:nth-child(2) .gsc_rsb_std+ .gsc_rsb_std').get_text()))

            i10_index.append(int(soup.select_one('tr:nth-child(3) .gsc_rsb_sc1+ .gsc_rsb_std').get_text()))
            i10_index_last_5_years.append(int(soup.select_one('tr:nth-child(3) .gsc_rsb_std+ .gsc_rsb_std').get_text())) # alternative to tr:nth-child(3) is tr~ tr+ tr
        except:
            citations.append(None)
            citations_last_5_years.append(None)

            h_index.append(None)
            h_index_last_5_years.append(None)

            i10_index.append(None)
            i10_index_last_5_years.append(None)

        # Get the number of co_authers for each scholar
        try:
            view_co_authors = browser.find_element_by_id("gsc_coauth_opn")
            if view_co_authors is not None:
                view_co_authors.click()
                time.sleep(2)
                co_authers = browser.find_elements_by_xpath("//div[@class='gsc_ucoar gs_scl']")
                number_of_collaborators.append(len(co_authers))
        except:
            try:
                authors_list = browser.find_elements_by_xpath("//ul[@class='gsc_rsb_a']")
                if authors_list is not None:
                    number_of_collaborators.append(len(browser.find_elements_by_xpath("//ul[@class='gsc_rsb_a']/li")))
            except:
                number_of_collaborators.append(0)

        num += 1
        # time.sleep(2)

    # Create dictionary of data and load into a dataframe 
    dataset = {
        'school_name': school_name,
        'scholars_name': scholar_page_name,
        'bu_email': bu_mail,
        'home_page_link': home_page_link,
        'total_publications': total_publications,
        'number_of_collaborators': number_of_collaborators,
        'total_citation': citations,
        'citations_last_5_years': citations_last_5_years,
        'h_index': h_index,
        'h_index_last_5_years': h_index_last_5_years,
        'i10_index': i10_index,
        'i10_index_last_5_years': i10_index_last_5_years,
        'specialization_area': area_of_specialization,
        'profile_url': profile_url
    }

    bu_scholars = pd.DataFrame(dataset)

    date_scraped = datetime.today().strftime('%Y-%m-%d %I-%M%p')

    bu_scholars.to_csv('{}_faculty_{}.csv'.format(school, date_scraped), index=False)

    browser.quit()
    time.sleep(5)

Extracting data for American University of Nigeria
Scraping page: 1...
Scraping page: 2...
Scraping page: 3...
Scraping page: 4...
Scraping page: 5...
Scraping page: 6...
Scraping page: 7...
Extracting scholar: 1 data
Extracting scholar: 2 data
Extracting scholar: 3 data
Extracting scholar: 4 data
Extracting scholar: 5 data
Extracting scholar: 6 data
Extracting scholar: 7 data
Extracting scholar: 8 data
Extracting scholar: 9 data
Extracting scholar: 10 data
Extracting scholar: 11 data
Extracting scholar: 12 data
Extracting scholar: 13 data
Extracting scholar: 14 data
Extracting scholar: 15 data
Extracting scholar: 16 data
Extracting scholar: 17 data
Extracting scholar: 18 data
Extracting scholar: 19 data
Extracting scholar: 20 data
Extracting scholar: 21 data
Extracting scholar: 22 data
Extracting scholar: 23 data
Extracting scholar: 24 data
Extracting scholar: 25 data
Extracting scholar: 26 data
Extracting scholar: 27 data
Extracting scholar: 28 data
Extracting scholar: 29 data
Extrac